# Named Entity Recognition

This notebook is based on the tutorial of [sklearn_crfsuite](https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html)

We start with loading all the necessary packages:
* **sklearn_crfsuite** for the implementation of the Conditional Random Field
* **pandas** for nice displaying of dataframes
* **eli5** for illustration of learned parameters

In [1]:
import sklearn_crfsuite
import eli5
import pandas as pd
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report

/opt/jupyterhub-venv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/jupyterhub-venv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


# Corpus

Within this notebook we're working on the Conll 2000 Corpus, which was obtained from https://github.com/Franck-Dernoncourt/NeuroNER/

First, we take a look at the data to inspect the data format.

In [2]:
def print_input_file(filename, lines=10):
    with open(filename, 'r') as file:
        for line_number, line in enumerate(file):
            if line_number >= lines:
                break;
            else:
                print(line.strip())
                
print_input_file('train.txt')

-DOCSTART- -X- -X- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O


Load the data into train and test corpora and inspecting the raw sentences.

In [3]:
def load_file(filename): 
    text = []
    with open(filename, 'r') as f:
        sentence = []
        for line in f:
            if line.startswith('-DOCSTART-') or len(line.strip()) == 0:
                if len(sentence) > 0:
                    text.append(sentence)
                sentence = []
            else:
                l = line.strip().split(' ')
                sentence.append((l[0], l[1], l[3]))
    return text

train = load_file('train.txt')
test = load_file('test.txt')

In [4]:
def print_text(corpus, amount=1):
    for sentence in corpus[:amount]:
        print([l[0] for l in sentence])

print_text(train,4)

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
['Peter', 'Blackburn']
['BRUSSELS', '1996-08-22']
['The', 'European', 'Commission', 'said', 'on', 'Thursday', 'it', 'disagreed', 'with', 'German', 'advice', 'to', 'consumers', 'to', 'shun', 'British', 'lamb', 'until', 'scientists', 'determine', 'whether', 'mad', 'cow', 'disease', 'can', 'be', 'transmitted', 'to', 'sheep', '.']


Inspecting the loaded data.

In [5]:
def print_sentence(corpus, idx=0):
    for token in corpus[idx]:
        print(token)
        
print_sentence(train)

('EU', 'NNP', 'B-ORG')
('rejects', 'VBZ', 'O')
('German', 'JJ', 'B-MISC')
('call', 'NN', 'O')
('to', 'TO', 'O')
('boycott', 'VB', 'O')
('British', 'JJ', 'B-MISC')
('lamb', 'NN', 'O')
('.', '.', 'O')


# Compute features

In order to allow the CRF to learn how to distinguish between different Named Entities, we have to compute features for the individual words.

For the time being, we select information about the word itself, such as
* the word
* the suffix of the word
* the shape of the word
* the POS tag
but also about the word before and after. In addition, we add information about whether the word is at the beginning or the end of the sentence.

In [6]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features

For the first word, the features look as follows

In [7]:
word2features(train[0], 0)

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word.isupper()': True,
 'word.istitle()': False,
 'word.isdigit()': False,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB'}

For the first sentence, the features look like:

In [8]:
def sentence2features(corpus, sent_idx):
    sentence_features = []
    for i in range(len(corpus[sent_idx])):
        sentence_features.append(word2features(corpus[sent_idx], i))
    return sentence_features

sentence2features(train, 0)    

[{'word.lower()': 'eu',
  'word[-3:]': 'EU',
  'word.isupper()': True,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'NNP',
  'postag[:2]': 'NN',
  'BOS': True,
  '+1:word.lower()': 'rejects',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'VBZ',
  '+1:postag[:2]': 'VB'},
 {'word.lower()': 'rejects',
  'word[-3:]': 'cts',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'VBZ',
  'postag[:2]': 'VB',
  '-1:word.lower()': 'eu',
  '-1:word.istitle()': False,
  '-1:word.isupper()': True,
  '-1:postag': 'NNP',
  '-1:postag[:2]': 'NN',
  '+1:word.lower()': 'german',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False,
  '+1:postag': 'JJ',
  '+1:postag[:2]': 'JJ'},
 {'word.lower()': 'german',
  'word[-3:]': 'man',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'JJ',
  'postag[:2]': 'JJ',
  '-1:word.lower()': 'rejects',
  '-1:word.istitle()': False,
  '-

## Computing features for the entire corpus

In [9]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

X_train = [sent2features(s) for s in train]
y_train = [sent2labels(s) for s in train]

X_test = [sent2features(s) for s in test]
y_test = [sent2labels(s) for s in test]

In [10]:
X_train[0]

[{'word.lower()': 'eu',
  'word[-3:]': 'EU',
  'word.isupper()': True,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'NNP',
  'postag[:2]': 'NN',
  'BOS': True,
  '+1:word.lower()': 'rejects',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'VBZ',
  '+1:postag[:2]': 'VB'},
 {'word.lower()': 'rejects',
  'word[-3:]': 'cts',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'VBZ',
  'postag[:2]': 'VB',
  '-1:word.lower()': 'eu',
  '-1:word.istitle()': False,
  '-1:word.isupper()': True,
  '-1:postag': 'NNP',
  '-1:postag[:2]': 'NN',
  '+1:word.lower()': 'german',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False,
  '+1:postag': 'JJ',
  '+1:postag[:2]': 'JJ'},
 {'word.lower()': 'german',
  'word[-3:]': 'man',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'JJ',
  'postag[:2]': 'JJ',
  '-1:word.lower()': 'rejects',
  '-1:word.istitle()': False,
  '-

# Training of the CRF

We train the CRF based on the computed features by use of gradient descent with elastic net regularisation.

In [11]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1, 
    c2=0.1, 
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(X_train, y_train);

CPU times: user 16.2 s, sys: 191 ms, total: 16.4 s
Wall time: 16.4 s


/opt/jupyterhub-venv/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

CRFsuite CRF models use two kinds of features: state features and transition features. Let's check their weights 
using eli5.explain_weights:

In [12]:
eli5.show_weights(crf, top=10)

/opt/jupyterhub-venv/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


# Predict and Evaluate

In [13]:
y_pred = crf.predict(X_test)

def show_sentence_results(corpus, prediction, idx):
    df = pd.DataFrame(corpus[idx])
    df['prediction'] = prediction[idx]
    return df

show_sentence_results(test, y_pred, 0)

,0,1,2,prediction
0,SOCCER,NN,O,O
1,-,:,O,O
2,JAPAN,NNP,B-LOC,O
3,GET,VB,O,O
4,LUCKY,NNP,O,B-LOC
5,WIN,NNP,O,I-LOC
6,",",",",O,O
7,CHINA,NNP,B-PER,B-LOC
8,IN,IN,O,O
9,SURPRISE,DT,O,O


In [14]:
y_pred_train = crf.predict(X_train)

show_sentence_results(train, y_pred_train, 0)

,0,1,2,prediction
0,EU,NNP,B-ORG,B-ORG
1,rejects,VBZ,O,O
2,German,JJ,B-MISC,B-MISC
3,call,NN,O,O
4,to,TO,O,O
5,boycott,VB,O,O
6,British,JJ,B-MISC,B-MISC
7,lamb,NN,O,O
8,.,.,O,O


In [15]:
print(flat_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       B-LOC       0.62      0.73      0.67      1668
      B-MISC       0.62      0.66      0.64       702
       B-ORG       0.74      0.46      0.56      1661
       B-PER       0.69      0.79      0.74      1617
       I-LOC       0.58      0.51      0.54       257
      I-MISC       0.28      0.64      0.39       216
       I-ORG       0.67      0.47      0.55       835
       I-PER       0.76      0.90      0.83      1156
           O       0.98      0.98      0.98     38323

    accuracy                           0.92     46435
   macro avg       0.66      0.68      0.66     46435
weighted avg       0.93      0.92      0.92     46435



In [16]:
labels = list(crf.classes_)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(flat_classification_report(y_test, y_pred, labels=sorted_labels))

              precision    recall  f1-score   support

           O       0.98      0.98      0.98     38323
       B-LOC       0.62      0.73      0.67      1668
       I-LOC       0.58      0.51      0.54       257
      B-MISC       0.62      0.66      0.64       702
      I-MISC       0.28      0.64      0.39       216
       B-ORG       0.74      0.46      0.56      1661
       I-ORG       0.67      0.47      0.55       835
       B-PER       0.69      0.79      0.74      1617
       I-PER       0.76      0.90      0.83      1156

    accuracy                           0.92     46435
   macro avg       0.66      0.68      0.66     46435
weighted avg       0.93      0.92      0.92     46435



In [17]:
from sklearn.metrics import confusion_matrix

def flatten(y):
    return [word_label for sentence_labels in y for word_label in sentence_labels]

truth = flatten(y_test)
pred = flatten(y_pred)

cm = confusion_matrix(truth, pred)
cm

array([[ 1220,    50,   133,   143,     1,     6,     5,     4,   106],
       [   53,   466,    29,    36,     1,    13,     6,     3,    95],
       [  333,    81,   756,   236,     2,    10,     5,     7,   231],
       [  158,    24,    25,  1276,     0,     9,    16,    10,    99],
       [    5,     1,     0,     1,   130,    23,    45,    41,    11],
       [    1,     4,     0,     3,     6,   139,    12,    22,    29],
       [   15,     3,     8,    30,    57,    69,   392,   156,   105],
       [    9,     0,     0,    16,    15,    20,    32,  1039,    25],
       [  165,   117,    73,   102,    13,   214,    71,    77, 37491]])

In [18]:
labels.remove('O')
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(flat_classification_report(y_test, y_pred, labels=sorted_labels))

              precision    recall  f1-score   support

       B-LOC       0.62      0.73      0.67      1668
       I-LOC       0.58      0.51      0.54       257
      B-MISC       0.62      0.66      0.64       702
      I-MISC       0.28      0.64      0.39       216
       B-ORG       0.74      0.46      0.56      1661
       I-ORG       0.67      0.47      0.55       835
       B-PER       0.69      0.79      0.74      1617
       I-PER       0.76      0.90      0.83      1156

   micro avg       0.66      0.67      0.66      8112
   macro avg       0.62      0.64      0.62      8112
weighted avg       0.67      0.67      0.66      8112

